# The PKI Secret Engine

The public key infrastructure (PKI) secrets engine in Vault enables you to issue certificates directly via the API. This is very useful if you require automation around your certificate management. In these cases Vault will act as a certificate authority (CA) and issue certificates based on a signing certificate it manages. 

## Use Cases

Typical use cases for the PKI secret engine are:

- Integration with other tooling in order to have certificates manages centrally (e.g. management for self-signed certificates of service meshes).
- As a full blown CA within your network which manages trusted certificates.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token -dev-listen-address="0.0.0.0:8200"
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Login to Vault

Let us login to Vault. We started a development version vor simplicity.

In [ ]:
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token

### Enabling the Secrets Engine

We will use the CLI to enable the PKI secrets engine. This can also be done via the UI. For information on this, refer to the `01-key-value.ipynb` module.

In [ ]:
vault secrets enable pki

### Tuning

An aspect that is quite central for the PKI engine is tuning of the secrets engine. Tuning engines is when its configuration is changed to better fit its use case. This is mostly interesting for the PKI engine as certificates tend to have different lifetime requirements as credentials. For instance, it is not uncommon that root certificates have a validity of up to 10 years. This would not be allowed by the default configuration of secrets engines.

Let us first retrieve the current configuration of the engine:

In [ ]:
vault read /sys/mounts/pki/tune

As we can see, the maximal lease time to live (TTL) is 768 hours, which corresponds to only 32 days (the default). Thus, if we were to issue certificates for longer than 32 days, their lifetimes would be cropped to the 32 days. This is of course not desired, as we might have certificates that should be valid longer. Let us tune the secrets engine:

In [ ]:
vault secrets tune -max-lease-ttl=87600h pki

### Certificate Authorities

There are now two ways to handle the certificate authority. Either we have a certificate authority that is fully managed by Vault, or we use Vault to issue trusted certificates by an external authority. Generally, having a fully managed authority has the advantage that the root certificate is fully contained within Vault and is close to impossible to leak. The problem is that we need to configure applications to then trust this root certificate. The advantage of using an external authority is that this is typically already trusted. Therefore applications will automatically trust the certificates issued by Vault.

In both cases, we will not signs leaf certificates directly by the authority's root. It is best practice to have an intermediate certificate that performs the signing. Thus the PKI structure looks as follows:

![](./assets/img/pki-structure.excalidraw.png)

Let us start with the simpler use case, where Vault manages the root certificate.

We can use the `pki/root/generate/internal` endpoint to generate an internal root. See [the official API documentation](https://developer.hashicorp.com/vault/api-docs/secret/pki#generate-root) for more details. Essentially, using the endpoint, we will generate a self-signed CA of type `internal`, meaning that the key is kept internal to Vault and will never be displayed. This is the most secure way of performing this, as it reduces changes of leaking the key.

Let us generate a root CA that is valid 10 years (87600 hours). Thus, create a certificate that:

- Is valid for 10 years (has a time-to-live - TTL of 87600h).
- Uses `example.com` as its common name.
- Creates an internal Vault issuer called `internal-root`.
- Names the key to `internal-root` as well.
- An has 4096 key bits (the key will be RSA by default).

Complete the below command and execute it. It will store the certificate under `internal-root.crt` on the file system.

In [ ]:
vault write -field=certificate pki/root/generate/internal \
     common_name="..." \
     issuer_name="..." \
     key_name="..." \
     key_bits=... \
     ttl=... | tee internal-root.crt

Let us inspect the certificate that was created, to make sure that it is what we expect:

In [ ]:
openssl x509 -in internal-root.crt -noout -subject -dates

### Checking Issuers and Keys in Vault

We can also check that the issuer and the key were correctly created in Vault. Go to "Secrets Engines" and check out the issuers and keys. There should be an `internal-root` respectively. Have a look around what information you can find on the issuers, the keys, and the currently issued certificates on the UI.

![](./assets/img/pki-issuer-list.png)

![](./assets/img/pki-key-list.png)

### Published Information

Note that when you click on the issuer and scroll down, you will see that there are no issuer URLs configured. Therefore, it will be impossible to find a reference to the CA when someone recieves a certificate issued by Vault. This is not ideal. Any issued certificate should always have a reference to a public URL where the CA can be found, and another where the certificate revocation list (CRL) can be found. This is important such that clients can validate the chain of trust, and ensure that the certificate they are seeing has not been revoked even though it is still within its validity period. Let's configure these URLs.

Note again that here we use the local address for Vault as this is a test environment. In practice it is very important that these URLs are reachable by any point in your network where a leaf certificate might be in use.

In [ ]:
vault write pki/config/urls \
     issuing_certificates="$VAULT_ADDR/v1/pki/ca" \
     crl_distribution_points="$VAULT_ADDR/v1/pki/crl"

### Creating an Intermediate Authority

Now that the root CA is ready, we would generate an intermediate authority. This would be the point where you would start if you would use an external root authority.

Since Vault 1.11.0, it is possible to host various issuers within the same PKI engine in Vault. However, tt is good practice to keep the intermediate separate from the root. Thus HashiCorp typically recommends to use a different secrets engine mount for the intermediate PKI engine. We will also do this here.

Let us enable a new PKI engine (at a different path), and tune it. Tune the PKI engine to support issuing certificate up to 3 years (26300h).

In [ ]:
vault secrets enable -path=pki-int pki
vault secrets tune -max-lease-ttl=... pki-int

### Creating a Signing Request

We will first need to generate a certificate signing request (CSR) for the intermediate. This will essentially generate an internally managed key by Vault and a CSR. This CSR can then be passed to the root to return a signed certificate. That certificate is then the intermediate CA's certificate, and must then be imported into the engine again.

Note that in the case we are using an external root CA, we would perform these exact steps, except that the signing of the CSR by the root CA would not be done by performing a REST call against Vault, but by passing that CSR to the external root CA.

The following commands are a bit more complex, so let us look at them in detail.

We will be using the `intermediate/generate/internal` API endpoint to generate an intermediate CA. Thus, it will not self-sign (like in the Root CA case) and therefore not return a signed certificate as before. It will instead return a CSR. The key is still kept private. Here we use JSON format as output and `jq` to retrieve the CSR into a file to persist it for simplicity.

Note we are using the new PKI engine under the `pki-int` path.

In [ ]:
vault write -format=json pki-int/intermediate/generate/internal \
     common_name="example.com Intermediate Authority" \
     issuer_name="example-dot-com-intermediate" \
     | jq -r '.data.csr' | tee pki-intermediate.csr

In [ ]:
# we can then inspect the CSR
openssl req -in pki-intermediate.csr -noout -subject

### Signing the CSR

Now let's sign the CSR by the root PKI. Here we will again be using the `pki` path, where the root PKI engine is located. We are using the `root/sign-intermediate` endpoint to sign the CSR of an intermediate. We will need to reference which issuer we want to use. Since we have only one issuer on the `pki` path, it would take that one by default. However, it is good practice to reference it explicitly. Remember that we called it `internal-root`. Then we pass the CSR and provide it with a TTL for the intermediate.

Sign the CSR with the following:
- Provide the correct issuer reference: `internal-root`.
- Provide the TTL for the intermediate: 3 years (26300h).

In [ ]:
vault write -format=json pki/root/sign-intermediate \
     issuer_ref="..." \
     csr=@pki-intermediate.csr \
     format=pem_bundle ttl="..." \
     | jq -r '.data.certificate' | tee intermediate.cert.pem

Perfect, we should now have our certificate under `intermediate.cert.pem`. Let's have a look.

In [ ]:
openssl x509 -in intermediate.cert.pem -noout -subject -dates

### Importing the Signed Certificate

We now need to import the signed certificate into the `pki-int` secrets engine. Let's do this:

In [ ]:
vault write pki-int/intermediate/set-signed certificate=@intermediate.cert.pem

The above command should have returned you a warning, that no authority information access (AIA) is contained in the CA. That is the point we made above regarding the CA distribution URL and the CRL URL. Let us configure these before we issue any certificates:

In [ ]:
vault write pki-int/config/urls \
     issuing_certificates="$VAULT_ADDR/v1/pki-int/ca" \
     crl_distribution_points="$VAULT_ADDR/v1/pki-int/crl"

### Creating Roles

Before leaf certificates can be issues by the intermediate, we will need to create roles. Roles are what define which types of certificates are allowed to be issued by this CA. Typically, clients will log in to Vault and then have a small set of roles for which they can generate certificates. For instance, an application that is known to be hosted under `my-app.example.com` will have a role which only allows it to generate certificates for that domain (and potentially sub-domains). That application should not be allowed to request a certificate for `super-secure.example.com`.

Let us create such a role which:

- Is allowed to use the intermediate (the default) issuer for certificates.
- Is allowed to generate certificates for the `my-app.example.com` domain.
- Is allowed to get bare domains, meaning exactly the `my-app.example.com` domain.
- Is allowed to get certificates for subdomains (such as `testing.my-app.example.com`).
- Is not allowed to get wildcard certificates (e.g. `*.my-app.example.com`).
- Can only have certificates that are valid at most 30 days (720 hours).
- Are not allowed to be used for client authentication.

For more information on everything that can be configured, check out [the API documentation](https://developer.hashicorp.com/vault/api-docs/secret/pki#create-update-role). This exact and yet very simple configuration of roles is what makes Vault very powerful to manage certificates.

In [ ]:
vault write pki-int/roles/my-app \
     issuer_ref="$(vault read -field=default pki-int/config/issuers)" \
     allowed_domains="..." \
     allow_bare_domains=... \
     allow_subdomains=... \
     allow_wildcard_certificates=... \
     max_ttl="..." \
     client_flag=...

### Issuing Certificates

Now everything is set up. Let us try to generate a set of certificates. Let us first generate a single certificate and make sure that everything is in order:

In [ ]:
vault write -field=certificate pki-int/issue/my-app \
    common_name="my-app.example.com" \
    ttl="24h" | tee leaf.pem

In [ ]:
# check the extensions, TTL, CN, etc
openssl x509 -in leaf.pem -noout -text

The output should correctly show only server authentication under `X509v3 Extended Key Usage`, the correct issuer including the CRL and CA issuer URIs. Perfect.

Now let us see if we can test the boundaries of the role ...

In [ ]:
# this should work
vault write -field=certificate pki-int/issue/my-app \
    common_name="super.duper.test.my-app.example.com" | tee subdomain.pem

In [ ]:
# this should NOT work (wildcard certificate)
vault write pki-int/issue/my-app \
    common_name="*.my-app.example.com"

In [ ]:
# this should NOT work (wrong domain)
vault write pki-int/issue/my-app \
    common_name="other-app.example.com"

In [ ]:
# lets test a certificate that is valid much longer than what we configured ...
vault write -field=certificate pki-int/issue/my-app \
    common_name="long-lived.my-app.example.com" \
    ttl="43800h" > longlived.pem
# check out the validity period of the certificate, it should still only be 30 days (cropped)
openssl x509 -in longlived.pem -noout -dates

### Issuing via the UI and Exploration

You can also check all issued certificates in the UI, view their information, and even revoke them:

![](./assets/img/pki-certificate-list.png)

![](./assets/img/pki-certificate-info.png)

### Generate Certificate

Moreover, you can also go on the roles, view their information, and then either generate a certificate or sign a CSR using that role. You can also use the SAN options to view all available options you might want to additionally request when generating a certificate directly in the UI.

![](./assets/img/pki-role-info.png)

![](./assets/img/pki-generate-certificate.png)

## ADVANCED: Other PKI Capabilities

Try to perform the following actions. In order to achieve this, consider the [official documentation](https://developer.hashicorp.com/vault/docs/secrets/pki) as well as the [API documentation](https://developer.hashicorp.com/vault/api-docs/secret/pki).

1. Revoke a (leaf) certificate that you issued before using the serial of the certificate via the CLI/API.
2. Force a rotation of the CRL signature.
3. Assuming that some certificates are expired in the CRL, perform a tidy operation on the intermediate CA.

Let us simulate a root roation. In such a case a new root CA needs to be used because the old root is expiring. There are several approaches to perform this while allowing clients to update their trust within a defined time window. We will choose an approach where this is done by cross-signing an intermediate CA by both root CAs, such that they are both valid at the same time. This allows clients to update their trust into the new CA until the old one expires.

In order to do this:

1. Create a new self-signed root CA that will be our new root CA.
2. Use the `cross-sign` API endpoint on the intermediate to generate a CSR for the new root CA (see https://developer.hashicorp.com/vault/api-docs/secret/pki#generate-intermediate-csr, you will need the `key_ref` field with the default issuer).
3. Sign the CSR with the new root CA.
4. Import the certificate into the intemediate using the `set-signed` API endpoint.
5. Issue a certificate with that issuer and validate its trust chain.
6. Optionally, update the trust chains on both issuers of the intermediate to use both roots (by patching the issuer configuations with a `manual_chain`).

For a solution on how to do this, check out the official tutorial steps 9 to 11: https://developer.hashicorp.com/vault/tutorials/pki/pki-engine#step-9-create-a-cross-signed-intermediate

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid
rm *.pem *.csr *.crt